# Using MLP and LSTM cells for Pulse Shape Analysis


## Introduction

The NEDA collaboration has build a multidetector to be used for Nuclear Physics.
It is composed of scintillators sensitive to **neutrons** but also to *gamma-rays* :
as the goal of the detector is to measure neutrons, it is then crucial to separate both type of particles !

The signal of scintillation produced and collected by the photo-multiplicator is different depending of the interacting particle. Indeed, the signal is characterized by a rising time and a decay time, the decay time depending of the nature of the particle.

The goal of this tutorial is to build neural networks able to efficiently tag the identified particle.
To perform such a task, signals out of the NEDA modules are fully digitized with a sampling of 10ns over a range of 730ns. 

## How to read signals from a ROOT file 

Import the library able to read a ROOT file which contains a set of signals used for training
(see https://github.com/scikit-hep/uproot if you don't have it installed)

In [ ]:
!pip install uproot
import uproot

More information about uproot at https://github.com/scikit-hep/uproot

Now, download the root file containing a set of signal consisting in entries in a ROOT TTree called training

In [ ]:
!rm DBofSignals.root*
! wget http://agata-ipnl.in2p3.fr/ftp/DBofSignals.root
tree = uproot.open("DBofSignals.root")['training']

To get some informations on the Tree itself : 

In [ ]:
print(tree)
print(tree.allkeys())

As you can see, there are different branches. For this tutorial, only the signals and truth are relevant.
To get some entries of the tree


In [ ]:
tree.arrays()

It can be seen 75 bin are used for each signal. Indeed, the last two bins are used to store different informations : 
* bin 73 contains the integral of the signal since the real signal is normalized before being ranged from bin 0 to 73

* bin 74 contains a TDC value i.e. the arrival time of the signal with respect to a reference (pulsed beam)


For each entry, the branch truth contains wether or not the signal correponds to a gamma-ray 0 or a neutron 1

Now plot the distribution of TDC, Integral and some signals corresponding to gamma-ray and neutrons


Plot the truth distribution to see the proportion of gammas and neutrons is equal

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

x_t = tree.array('truth')
plt.hist(x_t,color='green', bins=10)
plt.title('The gamma-neutron distribution')
plt.show()

Plot the distribution of the amplitude of all the signals

In [ ]:
x_s = tree.array('signals')
x_a = x_s[:,73]
plt.hist(x_a,color='blue', bins=1000)
plt.title('The Amplitude Distribution')
plt.show()

Plot the distribution of the TDC of all the signals

Plot and compare normalized signals for gamma and neutron

In [ ]:
x_i = tree.array('indices')
x_s = tree.array('signals')
n = 73
plt.figure(figsize=(30,10))
print('First signal of the DB is a gamma-ray',x_t[0])
plt.scatter(x_i[0,:n],x_s[0,:n], s=100, color ='red', alpha=0.5, label='Signal for a gamma-ray')
print('Second signal of the DB is a neutron',x_t[1])
plt.scatter(x_i[1,:n],x_s[1,:n], s=100, color ='blue', alpha=0.5, label='Signal for a neutron')
plt.xlabel('Time (10ns/bin)')
plt.ylabel('Normalized Amplitude')
plt.legend()
plt.show()

Here is the same plot but showing real signals i.e. not normalized

In [ ]:
plt.figure(figsize=(30,10))
x_tsg = x_s[0,73]*x_s[0]
x_tsn = x_s[1,73]*x_s[1]
plt.scatter(x_i[0,:n],x_tsg[:n], s=100, color ='red', alpha=0.7, label='Signal for a gamma-ray')
plt.scatter(x_i[1,:n],x_tsn[:n], s=100, color ='blue', alpha=0.5, label='Signal for a neutron')
plt.xlabel('Time (10ns/bin)')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

In [ ]:
x_tdc = x_s[:,74]
plt.hist(x_tdc,color='magenta', bins=100)
plt.title('The TDC Distribution')
plt.xlim(0,1)
plt.show()

## How to declare a Neural Network

Import the TensorFlow, Keras and numpy libraries :

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
import numpy as np
import sys

We create a model for a fully connected network (dense layers) with 4 layers :
* 75 neurons on the first layer (input)
* 10 neurons
* 4 neurons
* 2 neurons

In [ ]:
mlp = keras.Sequential()
mlp.add(keras.layers.Dense(10, activation='relu', input_shape=(75,)))
mlp.add(keras.layers.Dense(4, activation='relu'))
mlp.add(keras.layers.Dense(2, activation='softmax', name='output_layer'))

We use ReLU activation functions for the first layers and a softmax activation function for the last layer to unsure that the sum of the 2 neuron's values is 1.
We can ask for a description of our network :

In [ ]:
mlp.summary()

This gives us informations about the 3 layers we have created : 3 dense layers with 10, 4 and 2 neurons. The number of trainable parameters is given as well :
* 760 for the first layer (75 * 10 weights + 10 bias)
* 44 for the second layer (10 * 4 weights + 4 bias)
* 10 for the third layer (4 * 2 weights + 2 bias)

So a total of __814 parameters__ for our little model! 

Our Multi Layer Perceptron is complete, it is composed of one input layer of 75 neurons, 2 hidden layers of respectively 10 and 4 neurons and an output layer of 2 neurons. The idea is that the first output neuron will stand for *this is a gamma signal* and the second output neuron will stand for *this is a neutron signal*. Idealy, a gamma signal would output [1,0] and a neutron signal would output [0,1].

We could use our network right away, but as the weights and bias are currently random values chances are it will not be very effective. Statisticaly, it should give around 50% of good answers : as well as flipping a coin... To improve that, let's train our network!

## How to train the Neural Network

The idea is to feed to the network a signal along with the correct output. We will do so by batches of 100 signals+answers. At each step we will compute the error, ie the distance between the given answer and the provided truth. We will then slightly change the network parameters in order to reduce this distance and start again.

We need to provide informations for the training process :
* Which loss function to use (how do we compute the error)
* Which optimizer to use (how do we perform the back propagation)
* Which metrics should be computed at each step of the process

For example :

In [ ]:
mlp.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.SGD(), metrics=[keras.metrics.categorical_accuracy])

You can come back later to this line and tests other [loss functions](https://keras.io/losses/) and [optimizers](https://keras.io/optimizers/) and change their parameters.

We now need to format our data to be able to give input values and correct answers to the network:

In [ ]:
truth = tree.array("truth")
signals = tree.array("signals")

#Number of events we want to use
nb_events = len(truth)

#We separate data in 2 sets : 1 for training, the other one for final validation
input_data = signals[:nb_events-20000]
input_truth = tf.keras.utils.to_categorical(truth[:nb_events-20000])

validation_data = signals[nb_events-20000:nb_events]
validation_truth = tf.keras.utils.to_categorical(truth[nb_events-20000:nb_events])

We have our data: let's go! 
We give the input values and correct answers to the network plus :
* epochs : the number of times we want to run on all our data
* batch_size : how many values are used at once
* validation_split : percentage of data used for validation (and not for the training). After each epoch, the network will be tested on these data.

We will also use *callbacks* :
* EarlyStopping : Used to stop the training if the loss is not improving on the validation dataset for 10 epochs, even if we did not reach the set number of epochs (smells like overtraining...).
* Tensorboard : Used to log training info (see later)
* ModelCheckpoint : Used to save our model in a file each time we set a record on the validation dataset (keep our champion!).

In [ ]:
mlp.fit(input_data, input_truth, epochs=200, batch_size=100, validation_split=0.1, callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min'),keras.callbacks.TensorBoard("logs/mlp_neda"),keras.callbacks.ModelCheckpoint('mlp_neda.hdf5', monitor='val_loss', save_best_only=True, mode='min')])

## How to test the training

Now that our network is trained, we can test it on the 20 000 last signals (not used during training!):

In [ ]:
mlp=load_model("mlp_neda.hdf5") # This is our champion
score = mlp.evaluate(validation_data, validation_truth, batch_size=20000)
print("Proportion of correct predictions : ",score[1]*100,"%")
print("Loss : ",score[0])

## Playing with some parameters

You can now play with the different parameters to check how it affects the training time and the final accuracy of the network. Some ideas :
* you can use TensorBoard to have a look at the plots created during training and the graph of operations executed by the network : you should have a *TensorBoard* link at the bottom right corner of this page.
* number of different signals used
* size of the batch
* number of epochs
* activation functions used (relu?, sigmoïd?)
* loss function, gradient descent optimizer, ...

## Building a recursive network (Long Short Term Memory)

We will now create a second neural network to perform the same task, but this time using LSTM cells.

#### Building the network :
* We start by rounding the input values : this is not mandatory but might speed up the training
* LSTM layers need a 3D input vector : [nb_signals, nb_timesteps, nb_features] but our input is [nb_signals, nb_bins]. We reshape our vector (nb_signals, 75) in (nb_signals, 75, 1) : 75 timesteps of 1 value.

In [ ]:
state_size = 50

lstm = keras.Sequential()
lstm.add(keras.layers.Lambda(lambda x : keras.backend.round(x*100), input_shape=(75,)))
lstm.add(keras.layers.Reshape((75,1)))
lstm.add(keras.layers.LSTM(state_size))
lstm.add(keras.layers.Dense(2, activation='softmax', name='output_layer'))
lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[keras.metrics.categorical_accuracy])
lstm.summary()

#### Training :

We now have 10 502 parameters and a more complex network, epochs will take more time: start with small values!

In [ ]:
lstm.fit(input_data, input_truth, epochs=15, batch_size=200, validation_split=0.1, callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min'),keras.callbacks.TensorBoard("logs/lstm_neda"),keras.callbacks.ModelCheckpoint('lstm_neda.hdf5', monitor='val_loss', save_best_only=True, mode='min')])

#### Testing the trained network :

In [ ]:
lstm = load_model("lstm_neda.hdf5") # This is our champion
score = lstm.evaluate(validation_data, validation_truth, batch_size=20000)
print("Proportion of correct predictions : ",score[1]*100,"%")
print("Loss : ",score[0])

Great! I have roughly the same results with much more computing time... Why on hearth would I want that??

Let's play a bit with the signals: there are many reasons that could cause our signals to be not so perfectly synchronyzed : bad calibration, changing the length of a fiber, ... We will simulate these real life conditions by inserting some zero values at the begining of the signals.

### Creation of data sets with a shift of 1,2 and 3 bins

In [ ]:
print(validation_data.shape)
#Take the 72 first signals + 2 last ones
shift_1 = np.append(validation_data[:,:72],validation_data[:,73:75],axis=1)
#Add a zero at the begining
shift_1 = np.insert(shift_1,[0],0,axis=1)
print(shift_1.shape)
#Take the 71 first signals + 2 last ones
shift_2 = np.append(validation_data[:,:71],validation_data[:,73:75],axis=1)
#Add 2 zeros at the begining
shift_2 = np.insert(shift_2,[0,0],0,axis=1)
print(shift_2.shape)
#Take the 70 first signals + 2 last ones
shift_3 = np.append(validation_data[:,:70],validation_data[:,73:75],axis=1)
#Add 3 zeros at the begining
shift_3 = np.insert(shift_3,[0,0,0],0,axis=1)
print(shift_3.shape)

### Displaying the signals

In [ ]:
n = 75
nb_sig = 2000
indices = np.reshape(x_i.flatten(),(x_i.shape[0],75))
x_data = indices[:nb_sig,:n]
x_data=np.reshape(x_data,(x_data.shape[0]*n))

print("No shift")
y_data = validation_data[:nb_sig,:n]
y_data=np.reshape(y_data,(y_data.shape[0]*n))

plt.figure(figsize=(30,10))
plt.hist2d(x_data,y_data,(n,100))
plt.xlabel('Time (10ns/bin)')
plt.ylabel('Normalized Amplitude')
plt.show()

print("+1 bin shift")
y_data = shift_1[:nb_sig,:n]
y_data=np.reshape(y_data,(y_data.shape[0]*n))

plt.figure(figsize=(30,10))
plt.hist2d(x_data,y_data,(n,100))
plt.xlabel('Time (10ns/bin)')
plt.ylabel('Normalized Amplitude')
plt.show()

print("+2 bins shift")
y_data = shift_2[:nb_sig,:n]
y_data=np.reshape(y_data,(y_data.shape[0]*n))

plt.figure(figsize=(30,10))
plt.hist2d(x_data,y_data,(n,100))
plt.xlabel('Time (10ns/bin)')
plt.ylabel('Normalized Amplitude')
plt.show()

print("+3 bins shift")
y_data = shift_3[:nb_sig,:n]
y_data=np.reshape(y_data,(y_data.shape[0]*n))

plt.figure(figsize=(30,10))
plt.hist2d(x_data,y_data,(n,100))
plt.xlabel('Time (10ns/bin)')
plt.ylabel('Normalized Amplitude')
plt.show()


### Testing the networks on the different data sets

We now have 4 data-sets : the first one is very well synchronized with the training set, the others are respectively shifted by 1,2 and 3 bins. We'll check the evolution of our networks accuracy on the different data-sets.

In [ ]:
score_mlp=np.zeros(4)
score_mlp[0] = mlp.evaluate(validation_data, validation_truth, batch_size=20000)[1]
score_mlp[1] = mlp.evaluate(shift_1, validation_truth, batch_size=20000)[1]
score_mlp[2] = mlp.evaluate(shift_2, validation_truth, batch_size=20000)[1]
score_mlp[3] = mlp.evaluate(shift_3, validation_truth, batch_size=20000)[1]

score_lstm=np.zeros(4)
score_lstm[0] = lstm.evaluate(validation_data, validation_truth, batch_size=20000)[1]
score_lstm[1] = lstm.evaluate(shift_1, validation_truth, batch_size=20000)[1]
score_lstm[2] = lstm.evaluate(shift_2, validation_truth, batch_size=20000)[1]
score_lstm[3] = lstm.evaluate(shift_3, validation_truth, batch_size=20000)[1]


### Plotting the results

In [ ]:
print(score_mlp)
print(score_lstm)
plt.figure(figsize=(20,10))
plt.plot(score_mlp,label="MLP")
plt.plot(score_lstm,label="LSTM")
plt.xlabel('Shift (bins)')
plt.ylabel('Accuracy')
plt.legend()
plt.title("Evolution of accuracy with signal shift")
plt.show()

Because of the stochastic nature of the training results may vary, but you should be able to see a better robustness of the LSTM network to time shifts in the signals.